In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet


# загружаем тренировочный набор данных
file_path_train = r'train_dataset_train.csv'
df = pd.read_csv(filepath_or_buffer=file_path_train, sep=';',
                 parse_dates=['VISIT_MONTH_YEAR'])

# Количество уникальных значений по полям тренировочного набора данных
for key in df.keys():
    print(f'{key} = {df[key].nunique()}')
# тип данных по столбцам
df.info()

# преобразуем VISIT_MONTH_YEAR из object в datetime64
df['VISIT_MONTH_YEAR'] = '01.' + df['VISIT_MONTH_YEAR']
df['VISIT_MONTH_YEAR'] = df['VISIT_MONTH_YEAR'].str.split(pat='.')
l_list = []
for l in df['VISIT_MONTH_YEAR']:
    l = l[::-1]
    l = '.'.join(l)
    l = '20' + l
    l_list.append(l)
df['VISIT_MONTH_YEAR'] = l_list
df['VISIT_MONTH_YEAR'] = df['VISIT_MONTH_YEAR'].astype('datetime64')

# преобразуем train в сводную таблицу со столбцами по времени
table = pd.pivot_table(df, values='PATIENT_ID_COUNT',
                       index=['ADRES', 'MKB_CODE', 'PATIENT_SEX', 'AGE_CATEGORY'],
                       columns=['VISIT_MONTH_YEAR'], aggfunc=np.sum, fill_value=0)

index = table.index.get_level_values
# список, содержащий все сочетания полей в тренировочном наборе данных
z_list = list(zip(index(0), index(1), index(2), index(3)))

# загружаем тестовый набор данных
file_path_test = r'test_dataset_test.csv'
df_test = pd.read_csv(filepath_or_buffer=file_path_test, sep=';',
                      parse_dates=['VISIT_MONTH_YEAR'])

# Количество уникальных значений по полям тестового набора данных
for key in df_test.keys():
    print(f'{key} = {df_test[key].nunique()}')

# тип данных по столбцам
df_test.info()

states = df['ADRES'].unique() # перечень населенных пунктов в тренировочном наборе данных
states_test = df_test['ADRES'].unique() # перечень населенных пунктов в тренировочном наборе данных

# оставляем только записи по городам, которые есть в тестовом наборе данных
table = table.loc[states_test, :, :, :, :]

# проверка, все ли города из тестового набора данных есть в тренировочном
for adres in df_test['ADRES'].unique():
    print(f'{adres} is in {adres in states}')

#создаем список, содержащий все пересечения df_test и table по полям:
# 'ADRES', 'MKB_CODE', 'PATIENT_SEX', 'AGE_CATEGORY'
z_list_test = []
count = 0 # просто счетчик для прогресса
for i in df_test.index:
    count += 1
    if not count % 1000:
        print(count)
    k = (df_test.loc[i].ADRES, df_test.loc[i].MKB_CODE,
         df_test.loc[i].PATIENT_SEX, df_test.loc[i].AGE_CATEGORY)
    if k in z_list:
        z_list_test.append(k)

# оставляем в тернировочном наборе только заиси, пересекающиеся с тестовым
table = table.loc[z_list_test,:]

# суммируем данные помесячно по всем полям
data = table.sum().copy()

# переименновываем столбцы по требованию для пакета Prophet
data = data.reset_index(name='y')
data = data.rename(columns={"VISIT_MONTH_YEAR": "ds"})

# небольшая аугментация
s_pred = []
np.random.seed(1)
aug_matrix = np.random.uniform(-0.05,0.05,(51, 100))
for i in range(100):
    aug = aug_matrix[:, i]
    data_aug = data.copy()
    data_aug['y'] = data_aug['y'] * (1 + aug)
    
    # тренируем модель, предсказываем на период +1
    m = Prophet(seasonality_mode='multiplicative')
    m.fit(data_aug.loc[0:])
    future = m.make_future_dataframe(periods=1, freq='MS')
    fcst = m.predict(future)
    s_pred.append(fcst.iloc[-1].loc['yhat'])
    
s_pred = np.array(s_pred)

# среднее значение по предсказаниям по аугментированным данным
s_pred = np.mean(s_pred)

# распределяем предсказанную сумму по полям в пропорциях последнего месяца
pred = s_pred * table.loc[:, '2022-03-01'] / table.loc[:, '2022-03-01'].sum()

# предсказания для сабмита первый шаг
prediction = []
count = 0 # просто счетчик для прогресса
for i in df_test.index:
    count += 1
    if not count % 1000:
        print(count)
    k = (df_test.loc[i].ADRES, df_test.loc[i].MKB_CODE,
         df_test.loc[i].PATIENT_SEX, df_test.loc[i].AGE_CATEGORY)
    if k in z_list:
        prediction.append(int(pred.loc[k]))
    else:
        # в тестовом наборе есть записи, отсутствующие в тренировочном
        # присваиваем им значение 0
        prediction.append(int(0))

df_test["PATIENT_ID_COUNT"] = prediction

# предсказания для сабмита второй шаг (распределяем потерянную сумму от
# округления на первом шаге)
delta = -10
while df_test["PATIENT_ID_COUNT"].sum() < s_pred:
    df_test["PATIENT_ID_COUNT"] = prediction
    delta += 10
    for i in range(50):
        df_test["PATIENT_ID_COUNT"] = df_test["PATIENT_ID_COUNT"] *\
                                      (s_pred + delta) / df_test["PATIENT_ID_COUNT"].sum()
        df_test["PATIENT_ID_COUNT"] = df_test["PATIENT_ID_COUNT"].apply(np.floor)
        df_test["PATIENT_ID_COUNT"] = df_test["PATIENT_ID_COUNT"].astype('int64')

# в тестовом наборе есть записи, отсутствующие в тренировочном
# присваиваем им значение 1
prediction_1 = list(df_test["PATIENT_ID_COUNT"])
count = 0
for i in df_test.index:
    count += 1
    if not count % 1000:
        print(count)
    k = (df_test.loc[i].ADRES, df_test.loc[i].MKB_CODE,
         df_test.loc[i].PATIENT_SEX, df_test.loc[i].AGE_CATEGORY)
    if k not in z_list:
        prediction_1[i] = 1
df_test["PATIENT_ID_COUNT"] = prediction_1

# сохраняем результат
file_path_submit = r'submit.csv'
df_test.to_csv(path_or_buf=file_path_submit, sep=';', index=False)

# выводим график
plt.plot(data['ds'], data['y'])
plt.plot(fcst['ds'], fcst['yhat'])
plt.title('Суммарная заболеваемость населения\nв городах Калининградской обл.,\nвыбранных для прогнозирования')
plt.ylabel('человек')
plt.legend(['Реальные данные','Модель'], loc=2)
plt.show()